# Yelp API Restaurant Calls By City

For each city listed in cities.csv, return all restaurants found via yelp api and
extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id


From above data, also determine the following for each city:

- total count of restaurants
- distribution of categories (eg. 10% Pizza, 20% Sushi, etc.)
- ratio of price level options (ie. total count of pricier levels divided by count of lower levels)
- total count of restaurants above a given rating
- concentration of restaurants (ie. total count divided by sq. mi of city)


In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from cities.csv
- Build function that calls and writes restaurant data to new csv, given a city
- Run list of cities through function

In [5]:
# Create dataframe from csv file
cities_df = pd.read_csv("cities.csv", names=["City", "County", "Population", "Area (sq. mi)"])
cities_df

,City,County,Population,Area (sq. mi)
0,Alameda,Alameda,"73,812",10.61
1,Albany,Alameda,"18,539",1.79
2,American Canyon,Napa,"19,454",4.84
3,Antioch,Contra Costa,"102,372",28.35
4,Atherton,San Mateo,"6,914",5.02
5,Belmont,San Mateo,"25,835",4.62
6,Belvedere,Marin,"2,068",0.52


In [88]:
# Function returns up to 1000 restaurant listings for input city
def get_restaurants(city, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls
        params = {"term": "restaurants", "location":city + ", CA", "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Log history
        count += 1
        print(f"Processing query set {count} of approx 20 for {city}")
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Breaks if no entries exist in query
            if response["businesses"] == []:
                break
            else:
                
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        restaurant_data.append([biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], "", biz["categories"][0]["title"], 
                                                biz["id"]])
                    else:
                        restaurant_data.append([biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])
    
    
    # Write to csv
    with open('restaurant_data.csv', 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()

In [90]:
# Loop thru list of cities in cities_df and call get_restaurants fn
for city in cities_df["City"]:
    get_restaurants(city, api_key)

Processing query set 1 of approx 20 for Albany
Processing query set 2 of approx 20 for Albany
Processing query set 3 of approx 20 for Albany
Processing query set 4 of approx 20 for Albany
Processing query set 5 of approx 20 for Albany
Processing query set 6 of approx 20 for Albany


### Create Dataframe

- Read csv file
- Clean data

In [93]:
# Read csv file
restaurants_df = pd.read_csv("restaurant_data.csv", encoding = "ISO-8859-1", 
                            names=["Name", "Lat", "Lng", "Address", "Zip", "Rating", "# of Reviews", 
                                   "Price Level", "Category", "Yelp ID"])

# Replace NaN entries with blank string
restaurants_df = restaurants_df.fillna('')
restaurants_df

,Name,Lat,Lng,Address,Zip,Rating,# of Reviews,Price Level,Category,Yelp ID
0,Zaytoon Mediterranean Restaurant & Bar,37.890520,-122.297730,1133 Solano Ave,94706,4.5,319,$$,Mediterranean,g15dMYbefEL-ylCgk0MBbw
1,310 Eatery,37.892410,-122.299270,747 San Pablo Ave,94706,4.5,647,$$,Burgers,1ErPhzdCaoMVSHsXc9TOmQ
2,Juanita & Maude,37.890990,-122.298800,825 San Pablo Ave,94706,4.5,162,$$$$,American (New),jzmCjMb4nJscElnEgtY-Pw
3,Wojia Hunan Cuisine,37.889540,-122.298340,917 San Pablo Ave,94706,4.5,141,$$,Szechuan,shWuD8dJ5wbXppAzEybpgw
4,Bowl'd Korean Rice Bar,37.891116,-122.288295,1479 Solano Ave,94706,4.0,1087,$$,Korean,vD0mp-ZGHixwQrdzCcxuGw
5,El Mono,37.905186,-122.304549,10264 San Pablo Ave,94530,4.5,1307,$$,Peruvian,Uq3u_kbGVGLjEDVU45WLog
6,DaNang,37.890050,-122.298620,905 San Pablo Ave,94706,4.5,50,,Vietnamese,TyIzSjcr0z0jimwLeISMQg
7,Oori Rice Triangles,37.890733,-122.293740,1247 Solano Ave,94706,4.0,340,$,Japanese,UuC0xMHDZ3xKlbBKSvqSqQ
8,The Hot Shop,37.889980,-122.298590,909 San Pablo Ave,94706,4.5,330,$,Mexican,4llVHbcdMPfrZImiiF-D9w
9,Enoteca Mediterraneo,37.890930,-122.290530,1389 Solano Ave,94706,4.5,48,$$,Mediterranean,smSB17tWe_xDw9TFQ3mdug


In [97]:
### TO DO: total counts of restaurant and add to city df.
### Clean data, how to treat missing address, zip, price level, food trucks/mobile, low review counts

restaurants_df["Category"].value_counts()

Pizza                        19
Mexican                      17
Food Trucks                  15
Burgers                      13
Thai                         12
Chinese                      12
Japanese                     11
Indian                        9
Breakfast & Brunch            7
Vietnamese                    7
Coffee & Tea                  6
Bakeries                      5
American (New)                5
Sushi Bars                    5
Himalayan/Nepalese            5
Fast Food                     5
Sandwiches                    4
Szechuan                      4
Mediterranean                 4
Italian                       3
Brazilian                     3
French                        3
Cafes                         3
Hot Dogs                      3
Seafood                       3
Korean                        3
American (Traditional)        3
Vegan                         3
Latin American                2
Delis                         2
Soul Food                     2
Desserts